In [1]:
##########----------##########----------##########----------##########----------

# Libraries, setup, and general use objects

In [60]:
## import full libraries
import urllib.request as url
import pandas as pd
import numpy as np
import ipyparallel as ipp
import pickle
import datetime as dt

## import needed functons from libraries
from bs4 import BeautifulSoup
from os import listdir, mkdir
from os.path import isdir, isfile
from sklearn.metrics   import f1_score
from sklearn.neighbors import KNeighborsRegressor

## define settings
set_gather_data = False
set_sample_frac = 1 / 5
set_parallel_cores = {'Download': 3, 'Model':12}
valid_prefix = [69, 72, 74, 99]
use_col_list = {'DATE':str, 'LATITUDE':float, 'LONGITUDE':float,
    'ELEVATION':float, "HourlyDryBulbTemperature":float,
                "HourlyPrecipitation":float} # "HourlyRelativeHumidity":float,
set_geo_box = {'Lon':(-125, -60), 'Lat':(25, 50)}

## define simple timing function
def time_check(s = 'A'):
    print('Time Check: Point ' + s)
    print(dt.datetime.now())
time_check('A')

## read in general purpose input files
with open('A_Input/year_links.txt', 'r') as conn:
    year_links = conn.readlines()
    conn.close()
    
city_list = pd.read_excel('A_Input/city_list.xlsx')
city_list = city_list.rename({'lon':'Lon', 'lat':'Lat'}, axis = 1)

print(
    '\n' +\
    'WARNING: Copies of city_list.xlsx used in two different projects.\n' +\
    '         Treat the us_travels version as the master version.\n' +\
    '         Sync this travel_weather version to it.'
)

Time Check: Point A
2022-03-04 10:56:56.276555

         Treat the us_travels version as the master version.
         Sync this travel_weather version to it.


#### Generate directories as needed

In [3]:
def make_directories():
    all_dirs = ['A_Input', 'B_Process', 'C_Output']
    all_dirs = all_dirs + ['B_Process/downloads', 'B_Process/model_data']
    for i in all_dirs:
        if not isdir(i): mkdir(i)
        
make_directories()

# Import Data

#### extract_links

In [5]:
def extract_links(address):
    
    ## retrieve raw web page
    url_connect = url.urlopen(address)
    all_links = url_connect.read()
    url_connect.close()
    
    ## extract all links to csv files
    all_links = BeautifulSoup(all_links)
    all_links = all_links.find_all('a')
    all_links = [i.string for i in all_links if i.string.find('.csv') != -1]
    
    ## eliminate files unlikely to represent US weather stations
    def us_range(x, target_range = valid_prefix):
        try:
            int(x[0:2])
        except:
            return False
        if int(x[0:2]) in target_range:
            return True
        else:
             return False
    all_links = filter(us_range, all_links)
    
    ## remove url if files have already been downloaded
    already_downloaded = listdir('B_Process/downloads')
    valid_links = list()
    for i in all_links:
        if address[-5:-1] + '_' + i + '.gz' in already_downloaded:
            pass
        else:
            x = address[-5:-1] + '_' + i + '.gz'
            y = address + i
            valid_links.append((x, y))
            
    return valid_links

#### download_files

In [6]:
def download_files(links_ext, ucl = use_col_list):
    import pandas as pd
    try:
        the_csv = pd.read_csv(links_ext[1], usecols = list(ucl.keys()),
            parse_dates = ['DATE'], dtype = ucl)
    except:
        the_csv = pd.read_csv(links_ext[1], usecols = list(ucl.keys()),
            parse_dates = ['DATE'], dtype = str)
        for j in ucl.keys():
            if ucl[j] == float:
                the_csv[j] = pd.to_numeric(the_csv[j], errors = 'coerce')
        
    the_csv = the_csv.round(3)
    the_csv.to_csv('B_Process/downloads/' + links_ext[0], encoding = 'utf-8',
                    index = False)

#### Execute code

In [7]:
if set_gather_data:

    ## read in list of links to file directory pages for each year of the data
    year_links = read_year_links()

    ## iterative extract files from the links on each directory page
    for i in year_links:
        links_extracted = extract_links(i)
        if len(links_extracted) < 1: continue
    
        with ipp.Cluster(n = set_parallel_cores['Download']) as rc:
            par_processes = rc.load_balanced_view()
            par_result = par_processes.map_async(download_files, links_extracted)
            par_result.wait_interactive()
            final_result = par_result.get()
            del par_processes, par_result, final_result

# Refine Data

#### refine_data (and compile)

In [8]:
def refine_data(file_dir, segment, ucl = use_col_list,
                sample_fraction = set_sample_frac):
    
    ## generate roster of files
    list_files = listdir(file_dir)
    list_files = [i for i in list_files if i[0] != '.']
    
    ## filter files to specified segment
    def us_range(x, target_range = segment):
        try:
            x[0:7]
        except:
            return False
        if x[0:7] in target_range:
            return True
        else:
             return False
    list_files = filter(us_range, list_files)
    
    ## assemble files
    all_data = list()
    for i in list_files:
        file_iter = pd.read_csv(file_dir + '/' + i, parse_dates = ['DATE'],
                                dtype = ucl)
        
        ## deconstruct day/times
        file_iter['Day'] = file_iter['DATE'].dt.dayofyear
        file_iter['Hour'] = file_iter['DATE'].dt.hour
        file_iter = file_iter.drop('DATE', axis = 1)
        
        ## score weather
        file_iter['HourlyPrecipitation'] = file_iter[
            'HourlyPrecipitation'].fillna(0)
        file_iter['Temperate'] = (file_iter['HourlyDryBulbTemperature'] > 55) &\
            (file_iter['HourlyDryBulbTemperature'] < 75) &\
            (file_iter['HourlyPrecipitation'] < 0.4)
        file_iter['Temperate'] = file_iter['Temperate'].astype(int)
        file_iter = file_iter.drop(['HourlyDryBulbTemperature',
            'HourlyPrecipitation'], axis = 1)
        
        ## rename columns to make capitalization consistent
        file_iter = file_iter.rename(columns = {'LATITUDE':'Lat',
            'LONGITUDE': 'Lon', 'ELEVATION':"Elev"})
        
        ## sample data if specified
        if sample_fraction < 1:
            assert sample_fraction > 0
            sample_n = file_iter.shape[0] * sample_fraction
            sample_n = int(sample_n)
            sample_n = np.max([sample_n, int((0.5**2)/(0.05**2))])
            sample_n = np.min([sample_n, file_iter.shape[0]])
            file_iter = file_iter.sample(
                n = int(sample_n),
                weights = (file_iter['Temperate'] * 1) + 1
            )

        ## drop files outside the US's rough lat/lon box; append others
        if max(file_iter.Lat) > 25 and min(file_iter.Lat) < 50:
            if max(file_iter.Lon) < -60 and min(file_iter.Lon) > -130:
                all_data.append(file_iter)

    ## compile files and save
    if len(all_data) > 0:
        all_data = pd.concat(all_data, axis = 0)
        all_data.to_csv('B_Process/model_data/' + str(segment[0]) +\
            '_weather_data.csv.gz', index = False, encoding = 'utf-8')
        return all_data
    else:
        pass
        #print('WARNING: ' + str(segment[0]) + ' files contain no valid data')

#### load_model_data

In [9]:
def load_model_data(file_directory = 'B_Process/model_data'):
    all_data = listdir(file_directory)
    all_data = [i for i in all_data if i[-6:] == 'csv.gz']
    for i in range(len(all_data)):
        all_data[i] = pd.read_csv(file_directory + '/' + all_data[i])
    all_data = pd.concat(all_data, axis = 0)
    all_data.reset_index()
    return all_data

#### Execute Code

In [10]:
if set_gather_data:
    for i in valid_prefix:
        for j in range(2015, 2020):
            segment_iter = [str(j) + '_' + str(i)]
            refine_data('B_Process/downloads', segment = segment_iter)

model_data = load_model_data()

# Build Model

#### split data into train and test subsets

In [11]:
## split data into train and test subsets
model_data.loc[:, 'Split'] = np.random.binomial(
    n = 1, size = (model_data.shape[0],), p = 0.8).astype(bool)

#### model_weather

In [12]:
def find_closest_means(new_data, mod):
    new_data = new_data[['Lon', 'Lat', 'Day', 'Hour']]
    return mod.predict(new_data)

def model_weather(dat = model_data):
    
    ## split data into train and test data
    test_data = dat[~dat['Split']]
    dat = dat[dat['Split']]
    
    ## round off data and average for each rounded grid
    simple_dat = dat.drop(['Elev', 'Split'], axis = 1).round()
    simple_dat.loc[:, 'Day']  = np.ceil(simple_dat.loc[:, 'Day']  / 5) * 5
    simple_dat.loc[:, 'Hour'] = np.ceil(simple_dat.loc[:, 'Hour'] / 2) * 2
    simple_dat.loc[:, ['Lon', 'Lat', 'Day', 'Hour']] = simple_dat.astype(int)
    simple_dat = simple_dat.groupby(['Lon', 'Lat', 'Day', 'Hour']).mean()
    simple_dat = simple_dat.reset_index()
    
    ## train k nearest neighbor model
    knn_model = KNeighborsRegressor(weights = 'distance')
    knn_model = knn_model.fit(
                                simple_dat[['Lon', 'Lat', 'Day', 'Hour']],
                                simple_dat['Temperate']
                                )
    
    ## save model to file
    with open('B_Process/knn_model.pkl', 'wb') as conn:
        pickle.dump(knn_model, conn)
        conn.close()
    
    ## announce accuracy of predictor function
    ml_score = find_closest_means(test_data, knn_model) > 0.5
    ml_score = ml_score.astype(int)
    ml_score = f1_score(test_data['Temperate'].values, ml_score).round(3)
    print('Weather Model F1: ' + str(ml_score) + ' (Threshold = 0.5)')
    
    return knn_model

#### Execute Code

In [20]:
if isfile('B_Process/knn_model.pkl') and ~set_gather_data:
    with open("B_Process/knn_model.pkl", 'rb') as conn:
        weather_model = pickle.load(conn)
        conn.close()
else:
    weather_model = model_weather()

Option #1


# Model Routes

#### model_city

In [75]:
def model_city(pd_nxy, mod):
    
    ## warn if outside range
    geo_bound_check = [
        pd_nxy.Lon < max(set_geo_box['Lon']),
        pd_nxy.Lon > min(set_geo_box['Lon']),
        pd_nxy.Lat < max(set_geo_box['Lat']),
        pd_nxy.Lat > min(set_geo_box['Lat'])
        ]
    
    if all(geo_bound_check):
        pass
    else:
        print('WARNING: ' + pd_nxy.City + ' is outside the geographic' +\
             ' bounds used to create the model.  Results may be inaccurate.\n')
    
    ## expand data to include all days of the year and hours of the day
    full_data = np.meshgrid(np.arange(1, 366), np.arange(0, 24))
    full_data = pd.DataFrame(map(np.ravel, full_data)).T
    full_data.columns = ['Day', 'Hour']
    for i in pd_nxy.index:
        full_data[i] = pd_nxy[i]
    
    ## predict weather
    full_data['Temperate'] = find_closest_means(full_data, mod)
    return full_data

0.5055898290579098

#### model_routes

In [138]:
def model_route(route, cl = city_list, mod = weather_model):
    
    ## extract coordinates of cities on the route
    pd_nxy = cl.loc[cl.Route == route, ['City', 'Lon', 'Lat']]
    
    ## predict weather conditions for each set of coordinates
    route_weather = []
    for i in range(pd_nxy.shape[0]):
        city_iter = pd_nxy.iloc[i,:]
        route_weather.append(model_city(city_iter, mod))
    route_weather = pd.concat(route_weather, axis = 0)
    
    ## score each day for temperate weather across routes
    i = route_weather.Hour
    route_weather = route_weather[(i > 7) & (i < 23)]
    route_weather = route_weather.drop(['Lon', 'Lat', 'City', 'Hour'], axis = 1)
    route_weather = route_weather.groupby('Day').mean().round(2)
    route_weather = route_weather.reset_index()

    ## inject route name into dataset and reorder
    route_weather['Route'] = route
    i = ['Route', 'Day', 'Temperate']
    route_weather = route_weather[i]
    
    return route_weather

#### find_best_month

In [143]:
def find_best_month(rm):
    
    ## calculate moving 30 day averages
    
    ## find best 30d period in each half of the year for each roupte
    rm['Half'] = rm['Day'] < 182
    #rm['Best'] = 
    
    print(rm)

find_best_month(routes_modeled)

               Route  Day  Temperate
0    California Plus    1       0.48
1    California Plus    2       0.48
2    California Plus    3       0.48
3    California Plus    4       0.48
4    California Plus    5       0.49
..               ...  ...        ...
360    Florida State  361       0.69
361    Florida State  362       0.69
362    Florida State  363       0.57
363    Florida State  364       0.57
364    Florida State  365       0.57

[5475 rows x 3 columns]


#### execute code

In [137]:
## Score average temperateness for all cities across each route
routes_modeled = []
for i in set(city_list.Route):
    routes_modeled.append(model_route(i))
routes_modeled = pd.concat(routes_modeled)

## determine the best 30 days routes for each route in each half of the year

In [ ]:
## Spare parts (approach will be folded into the new find_best_month function)
if False:
    ## score all 30 ranges of days for temperate weather
    route_weather['MovingMean30d'] = np.nan
    for i in np.arange(1, 366):
        j = np.arange(i - 15, i + 16) % 365
        j = route_weather.Day.isin(j)
        j = route_weather.loc[j, 'Temperate'].mean()
        route_weather.loc[i - 1, 'MovingMean30d'] = j
    
    ## find best 30 range in each half of the year
    route_weather['Best30d'] = route_weather.Day >= 182
    route_weather.Best30d = route_weather.groupby('Best30d').MovingMean30d\
        .transform(max)
    route_weather.Best30d = route_weather.Best30d == route_weather.MovingMean30d
    

# Render Dashboard

In [15]:
## TODO: centralize geographic boundary box setting
time_check('Z')

Time Check: Point Z
2022-03-04 09:43:21.591557
